In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [8]:
# This function parse the ouput of PerkinElmer 
# Scan Array CSV File to find the start and end 
# of the output table
# It also find the cyanine 5 channel (CH1, CH2 or CH3)
# that contains
# the SH2 domain reading 
def parse_text(start_line='BEGIN DATA', 
               out_file_path=''):
    a=0
    start = 0
    end = 0
    for l in open(out_file_path):
        a+=1
        if l.startswith(start_line):
            start += a
        if l.startswith('END DATA'):
            end += a
        if 'Cyanine 5' in l:
            ch = l.split(',')[1]
            ch = ch.replace('H','h')
    return start, end, ch

In [9]:
out_file_path = os.path.join('data_file_post','714-27_FRK.csv')
start, end , ch = parse_text(out_file_path=out_file_path)
print(start, end , ch)

70 19272 Ch1


In [10]:
#now we can read an output file
df = pd.read_table(out_file_path, sep=',',
                   skiprows=start, engine ='python', 
                   skipfooter=1)
df.head()

,Index,Array Row,Array Column,Spot Row,Spot Column,Name,ID,X,Y,Diameter,...,Ch1 SD,Ch1 B Median,Ch1 B Mean,Ch1 B SD,Ch1 % > B + 1 SD,Ch1 % > B + 2 SD,Ch1 F % Sat.,Ch1 Median - B,Ch1 Mean - B,Ch1 SignalNoiseRatio
0,1,1,1,1,1,GNEPDKYPCNGQV,>Part3-5146,3435,5436,200,...,21.26,181,185,21.31,23.0,5.2,0,3,5,8.63
1,2,1,1,1,2,NLSQGGYEELERV,>Part3-5151,3681,5436,200,...,291.77,196,195,18.25,66.0,44.9,0,32,186,12.50
2,3,1,1,1,3,ALAAEGYYDIRRV,>Part3-5166,3881,5429,200,...,21.60,199,199,28.87,17.3,1.1,0,5,6,7.07
3,4,1,1,1,4,QHDEPCYWRLTRV,>Part3-5187,4103,5429,200,...,23.90,189,189,19.18,56.9,31.1,0,26,25,11.21
4,5,1,1,1,5,EAGKPIYSRYGSV,>Part3-5199,4336,5444,200,...,19.97,186,187,17.73,25.6,8.1,0,3,4,10.66


In [12]:
# this are probably the useful columns 
# in the output file
suffixes = [' Median',' Mean',' SD',' B Median',
         ' B Mean',' B SD',' % > B + 1 SD',' % > B + 2 SD',
         ' F % Sat.',' Median - B',' Mean - B',' SignalNoiseRatio']
# to wich we add the peptide sequence and batch identifier
cols = ['Name','ID']+[ch + n for n in suffixes]
df = df[cols]
df.head()

,Name,ID,Ch1 Median,Ch1 Mean,Ch1 SD,Ch1 B Median,Ch1 B Mean,Ch1 B SD,Ch1 % > B + 1 SD,Ch1 % > B + 2 SD,Ch1 F % Sat.,Ch1 Median - B,Ch1 Mean - B,Ch1 SignalNoiseRatio
0,GNEPDKYPCNGQV,>Part3-5146,184,186,21.26,181,185,21.31,23.0,5.2,0,3,5,8.63
1,NLSQGGYEELERV,>Part3-5151,228,382,291.77,196,195,18.25,66.0,44.9,0,32,186,12.50
2,ALAAEGYYDIRRV,>Part3-5166,204,205,21.60,199,199,28.87,17.3,1.1,0,5,6,7.07
3,QHDEPCYWRLTRV,>Part3-5187,215,214,23.90,189,189,19.18,56.9,31.1,0,26,25,11.21
4,EAGKPIYSRYGSV,>Part3-5199,189,190,19.97,186,187,17.73,25.6,8.1,0,3,4,10.66


In [17]:
# The array is composed of 3 identical blocks 
# of 6400 spots, So that each spot on 
# the array is repeated 3 times.
# A spot can contain a peptide or a control (dye, Marker, GST...)
# A peptide can be spotted multiple times on the block (count > than 3)
# For example RVKEEGYELPYNP and GEEEHVYSFPNKQ are
# spotted 3 times in different position of the block
df['Name'].value_counts().head(10)

<<Marker>>       384
pY                72
GST               12
IgG               12
Triple Flaq       12
TAMRA             12
IgM               12
FITC              12
RVKEEGYELPYNP      9
GEEEHVYSFPNKQ      9
Name: Name, dtype: int64

In [26]:
# The processing of the data is arbitray. I can suggest:
# Median of the 3  replicated blocks
# Treat each peptide batch independently (ID) as we 
# observed that the quality of the peptide is different 
# between batches
df.groupby('ID')['Ch1 Median - B'].median().sort_values().tail()

ID
>Jerini-1477          14807.0
>Part2-P51812-356-    15223.0
>Part3-3260           16144.0
>Part3-2392           21341.0
>Part3-3106           23848.0
Name: Ch1 Median - B, dtype: float64

In [27]:
df.groupby('ID')['Ch1 Median - B'].median().sort_values().head()

ID
>Part3-3640       -121.0
>Part3-1033 SRC    -40.0
>Part3-5539        -31.0
>Part3-3661        -27.0
>Part3-5549        -14.0
Name: Ch1 Median - B, dtype: float64

In [ ]:
# cheereo